In [33]:
import numpy as np
import pandas as pd
import os 
import shutil
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [34]:
movies = pd.read_csv('.\our_dataset\movies.csv',sep=';',encoding='latin-1').drop('Unnamed: 3',axis=1)
print('Shape of this dataset :',movies.shape)
movies.head()

Shape of this dataset : (3883, 3)


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [35]:
ratings = pd.read_csv('.\our_dataset\\ratings.csv',sep=';') 
ratings = ratings.drop('timestamp',axis=1)
print('Shape of this dataset :',ratings.shape)
ratings.head()

Shape of this dataset : (1000209, 3)


,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [36]:
users = pd.read_csv('.\our_dataset\\users.csv',sep=';')
users = users.drop('zip-code',axis=1)
users = users.drop('occupation',axis=1)
print('Shape of this dataset :',users.shape)
users.head()

Shape of this dataset : (6040, 3)


,userId,gender,age
0,1,F,1
1,2,M,56
2,3,M,25
3,4,M,45
4,5,M,25


In [37]:
check_new = input('Are you a new user? (y/n) : ') 
if check_new == 'y' or 'Y': 
    print(users.shape)
    name = input('Enter your name : ') 
    age = input('Enter your age : ')
    gender = input('Enter your gender(M/F) : ')  
 
    user = pd.DataFrame({'userId' : users.shape[0] +1, 'gender' : gender, 'age' : int(age)}, index = [0]) 
    users = pd.concat([users, user], ignore_index=True)   
    print(users.shape) 
    print(users.tail())
    

(6040, 3)
(6041, 3)
      userId gender  age
6036    6037      F   45
6037    6038      F   56
6038    6039      F   45
6039    6040      M   25
6040    6041      F   20


In [38]:
print(users.shape)
print(users)

(6041, 3)
      userId gender  age
0          1      F    1
1          2      M   56
2          3      M   25
3          4      M   45
4          5      M   25
...      ...    ...  ...
6036    6037      F   45
6037    6038      F   56
6038    6039      F   45
6039    6040      M   25
6040    6041      F   20

[6041 rows x 3 columns]


In [52]:
class Recommender_new_user:
    def __init__(self, movies, ratings, users):
        # This list will stored movies that called atleast ones using recommend_on_movie method
        self.hist = [] 
        self.ishist = False # Check if history is empty 
        self.movies = movies
        self.ratings = ratings
        self.users = users.copy()
        self.nn_algo = NearestNeighbors(metric='cosine')
        self.users["gender"] = np.where(self.users["gender"] == "F", 0, 1)
        # self.users = self.users.drop('gender',axis=1)

    def fit(self):
        self.nn_algo.fit(self.users)
        return self.nn_algo

    def recommend_movies(self, user_id, n_recommend=10):
        # distance, neighbors = self.nn_algo.kneighbors([self.users.loc[self.users.userId == user_id]], n_neighbors=n_recommend+1)
        distance, neighbors = self.nn_algo.kneighbors(self.users.loc[self.users.userId == user_id], n_neighbors=n_recommend+1)

        userids = [self.users.iloc[i].name for i in neighbors[0]]
        recommends = [movieId for uid in userids for movieId in self.ratings.loc[(self.ratings.userId == uid) & (self.ratings.rating == 5)].dropna()['movieId']]
        recommends = np.array(recommends)
        np.random.shuffle(recommends)
        rec_movies = [str(self.movies.loc[self.movies.movieId == movieId].title.values[0]) for movieId in recommends]
        return rec_movies[:n_recommend]


In [46]:
print(users.shape)

(6041, 3)


In [54]:
recommender_new = Recommender_new_user(movies, ratings, users) 
recommender_new.fit() 
initial_movies = recommender_new.recommend_movies(6023, n_recommend=10)
print(recommender_new.recommend_movies(6023, n_recommend=10)) 

['Star Wars', 'Lord of the Flies (1963)', 'Out of Africa (1985)', 'Hoosiers (1986)', 'Shadow of a Doubt (1943)', 'Pulp Fiction (1994)', 'Cape Fear (1962)', "Breakfast at Tiffany's (1961)", 'Air Force One (1997)', 'Star Wars']
